# 1 探索数据


数据集使用 kaggle 数据集，解压后存放在images目录下
- 训练数据集路径：
images/train
- 测试数据集路径：
images/test

# 1.1 整理数据集
 
这里由于 trian 文件夹下的猫和狗的数据是混在一起的，需要将猫和狗的图片分别存储。

图片的命名规则：类别.编码.jpg

**注意：这里的处理只需要运行一次即可**


In [ ]:
import sys
print(sys.path)
# sys.path.append("/home/ubuntu/anaconda3/envs/dog-project/lib/python3.6/site-packages")
# sys.path.append("/home/ubuntu/anaconda3/pkgs/tensorflow-gpu-base-1.3.0-py36cuda8.0cudnn6.0_0/lib/python3.6/site-packages")
# sys.path.append("/home/ubuntu/anaconda3/pkgs/tensorflow-gpu-base-1.3.0-py36cuda8.0cudnn6.0_0/lib/python3.6/site-packages/tensorflow/contrib/keras/api")
# sys.path.append("/home/ubuntu/anaconda3/pkgs/tensorflow-gpu-base-1.3.0-py36cuda8.0cudnn6.0_0/lib/python3.6/site-packages")
# print(sys.path)



In [1]:
import os
import shutil
import time
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from scipy.stats import mode as spmode
import platform
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3,preprocess_input
from keras.layers import GlobalAveragePooling2D,Dense
from keras.models import Model
from keras.utils.vis_utils import plot_model
from keras.optimizers import Adagrad
from sklearn.datasets import load_files
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
import h5py
from sklearn.utils import shuffle
from keras.models import *
from keras.layers import *

np.random.seed(2017)


print(platform.system())

notmatch_path='images/notmatchs'
notmatch_txt = 'notmatchs.txt'
all_path = 'images/all'
train_path = 'images/all/trainnew'
dog_path = 'images/all/trainnew/dogs'


testnew_path = 'images/all/testnew'
test_path = 'images/all/testnew/test'


Using TensorFlow backend.


Linux


In [ ]:
# 下载特征向量
# !wget -P saved_models/  https://github.com/ypwhs/dogs_vs_cats/releases/download/gap/gap_InceptionV3.h5
# !wget -P saved_models/  https://github.com/ypwhs/dogs_vs_cats/releases/download/gap/gap_ResNet50.h5
# !wget -P saved_models/  https://github.com/ypwhs/dogs_vs_cats/releases/download/gap/gap_Xception.h5


In [2]:
# windows分隔符
splitflag = "\\"
if (platform.system() == 'Linux'):
    splitflag = "/"

#处理的样本量
train_limitcount = 12500
test_limitcount = 12500 #不限制个数全部使用

def create_dir(path):
    '''
    创建文件夹
    '''
    if (os.path.exists(path) == False):
        os.mkdir (path)
    else:
        print('文件夹已经存在：%s' % path)
        shutil.rmtree(path)
        os.mkdir (path)

# 限制训练文件数量,0不限制
def copy_image_bytype(srcpath, destpath, limitcount = 0):
    '''
    将猫狗文件分别拷贝到不同文件夹下
    '''
    
    filenames = [item.split(splitflag)[-1] for item in sorted(srcpath)]
    
    count = 0
    for src,name in zip(srcpath,filenames):
        dst = destpath + "/" + name
        if (os.path.exists(dst) == False):
            shutil.copyfile(src, dst)

        count += 1
        
        # 跳出循环
        if (limitcount == count):
            break
            
    return count

    

  

In [3]:
# 创建文件夹
create_dir(notmatch_path)
create_dir(all_path)

create_dir(train_path)
create_dir(dog_path)


create_dir(testnew_path)
create_dir(test_path)


cat_path = 'images/all/trainnew/cats'
create_dir(cat_path)

文件夹已经存在：images/notmatchs
文件夹已经存在：images/all


In [4]:
# 处理狗
dogs_all = glob("images/train/dog.*")

# 整理狗数据
count = copy_image_bytype(dogs_all, dog_path, train_limitcount)

print('There are %d total dog categories.' % count)


There are 12500 total dog categories.


In [5]:
# 处理猫
cats_all = glob("images/train/cat.*")

# 整理猫数据
count = copy_image_bytype(cats_all, cat_path, train_limitcount)

print('There are %d total cat categories.' % count)


There are 12500 total cat categories.


In [6]:
# 处理测试数据
test_all = glob("images/test/*")

# 整理猫数据
count = copy_image_bytype(test_all, test_path, test_limitcount)

print('There are %d total test categories.' % count)


There are 12500 total test categories.


In [7]:
#当前使用的样例
dogs_list = glob(dog_path+'/dog.*')
print(len(dogs_list))

test_list = glob(test_path+'/*')
print(len(test_list))

cats_list = glob(cat_path+'/cat.*')
print(len(cats_list))

12500
12500
12500


## 1.2 数据可视化

In [10]:
from PIL import Image

def get_filesizes(paths):
    '''
    获取文件大小
    '''
    filesizes = []
    for path in paths:
        size = os.path.getsize(path)
        filesizes.append(size)
    return filesizes

def get_filepixs(paths):
    '''
    获取像素：长和宽
    '''
    filepixs = []
    xpixs = []
    ypixs = []
    for path in paths:
        img = Image.open(path)
        pix = img.size #图片的长和宽
        xpixs.append(pix[0]) #图片的宽
        ypixs.append(pix[1]) #图片的长
        if (pix[0] > 1000):
            print(path)
    filepixs.append(xpixs)
    filepixs.append(ypixs)    
    
    return filepixs
    
def draw_hist(mylist, title, xlabel, ylabel, xmin, xmax, ymin, ymax):
    '''
    绘制直方图，参数依次为list,抬头,X轴标签,Y轴标签,XY轴的范围
    '''
    plt.hist(mylist, 100)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.show()
    
def draw_scatter(x1, y1, title, label):
    '''
    绘制像素的散点图
    '''
    plt.scatter(x1, y1, marker = 'x',color = 'red', s = 40 ,label = label)
    plt.xlabel('wide')
    plt.ylabel('length')
    plt.title(title)
    
#     plt.xlim(-1.5, 1.5)
#     plt.xticks(())  # ignore xticks
#     plt.ylim(-1.5, 1.5)
#     plt.yticks(())  # ignore yticks

    plt.show()   

    
def show_filesize(paths):
    '''
    展示文件大小分布
    '''
    filesizes = get_filesizes(paths)

    # train 中文件的的最小值、最大值、中位数、众数
    train_min = np.min(filesizes)
    print('train 中文件的的最小值:', train_min)
    train_max = np.max(filesizes)
    print('train 中文件的的最大值:', train_max)
    train_median = np.median(filesizes)
    print('train 中文件的的中位数:', train_median)
    train_mode = spmode(filesizes)
    print('train 中文件的的众数:', train_mode)
        
    title = 'file size distribute'
    xlabel = 'file size'
    ylabel = 'file count'
    xmin = np.min(filesizes)
    xmax = np.max(filesizes)
    ymin = 0
    ymax = 1200
    draw_hist(filesizes, title, xlabel, ylabel, xmin, xmax, ymin, ymax)

# 训练数据集文件大小的分布
# show_filesize(dogs_list)        
# 测试数据集文件大小的分布
# show_filesize(cats_list)          
    
def show_filepix(paths):
    '''
    显示像素分布
    '''
    filepixs = get_filepixs(paths)
    title = 'file pix distribute'
    draw_scatter(filepixs[0], filepixs[1], title, 'dog')    

ImportError: No module named 'PIL'

In [9]:
# 狗的数据集图片大小分布

show_filepix(dogs_list)

NameError: name 'Image' is not defined

In [ ]:
# 猫的数据集图片大小分布
show_filepix(cats_list)

## 1.4 数据预处理

### 1.4.1 异常数据清理

异常数据有:

标签与图片信息不相符的，例如：虽然标识了是狗的图片，但是图片中没有狗。

照片的背景过于复杂，识别起来很困难，例如：虽然有狗但是有很多干扰元素。

这里选择4种算法:ResNet50, VGG19, Xception, InceptionV3 获取的并集进行数据清理



In [ ]:
from keras.preprocessing import image


#可能概率前几个类型
topparma = 50
# 筛选的数量
doglimit = train_limitcount
catlimit = train_limitcount


# 对 ImageNet 分类进行分析
# 狗分类：118种，猫分类：7种

ImageNetdogs = [
 'n02085620','n02085782','n02085936','n02086079'
,'n02086240','n02086646','n02086910','n02087046'
,'n02087394','n02088094','n02088238','n02088364'
,'n02088466','n02088632','n02089078','n02089867'
,'n02089973','n02090379','n02090622','n02090721'
,'n02091032','n02091134','n02091244','n02091467'
,'n02091635','n02091831','n02092002','n02092339'
,'n02093256','n02093428','n02093647','n02093754'
,'n02093859','n02093991','n02094114','n02094258'
,'n02094433','n02095314','n02095570','n02095889'
,'n02096051','n02096177','n02096294','n02096437'
,'n02096585','n02097047','n02097130','n02097209'
,'n02097298','n02097474','n02097658','n02098105'
,'n02098286','n02098413','n02099267','n02099429'
,'n02099601','n02099712','n02099849','n02100236'
,'n02100583','n02100735','n02100877','n02101006'
,'n02101388','n02101556','n02102040','n02102177'
,'n02102318','n02102480','n02102973','n02104029'
,'n02104365','n02105056','n02105162','n02105251'
,'n02105412','n02105505','n02105641','n02105855'
,'n02106030','n02106166','n02106382','n02106550'
,'n02106662','n02107142','n02107312','n02107574'
,'n02107683','n02107908','n02108000','n02108089'
,'n02108422','n02108551','n02108915','n02109047'
,'n02109525','n02109961','n02110063','n02110185'
,'n02110341','n02110627','n02110806','n02110958'
,'n02111129','n02111277','n02111500','n02111889'
,'n02112018','n02112137','n02112350','n02112706'
,'n02113023','n02113186','n02113624','n02113712'
,'n02113799','n02113978']

ImageNetcats=[
'n02123045','n02123159','n02123394','n02123597'
,'n02124075','n02125311','n02127052']

In [ ]:
def get_not_animalarray(model, top, animalarray,img_paths, xsize, ysize, preprocess_input, decode_predictions):
    '''
    清理数据集中不是对应标签的文件
    model:模型
    top:可能概率前几个类型
    animalarray: ImageNet分类集
    img_paths:数据集路径
    x:图片宽
    y:图片高
    '''
    animals = []
    for img_path in img_paths:
        img = image.load_img(img_path, target_size=(xsize, ysize))
        imgarray = image.img_to_array(img)
        imgarray = np.expand_dims(imgarray, axis=0)
        imgarray = preprocess_input(imgarray)

        preds = model.predict(imgarray)
        predarray = decode_predictions(preds, top=top)
#         sorted(tmparray, key=lambda tmp: tmp[2], reverse=True) 

        # 判断是否有指定类型存在
        existflag = False
        for n in range(top):
            animal = predarray[0][n][0]
            if animal in animalarray:
                existflag = True
                break
        if existflag == False:
            animals.append(img_path)
            
    return animals

# [[('n02092002', 'Scottish_deerhound', 0.21568948),('n02097130', 'giant_schnauzer', 0.074100845)]]

#### 利用ResNet50网络进行ImageNet分类

In [ ]:
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions

ResNet50_model = ResNet50(weights='imagenet')

print(doglimit)
print(catlimit)

# # 不是狗的
# ResNet50_notdogs = get_not_animalarray(ResNet50_model, topparma, ImageNetdogs, dogs_list[0:doglimit], 224, 224, preprocess_input, decode_predictions)
# print(ResNet50_notdogs)

# # 不是猫的
# ResNet50_notcats = get_not_animalarray(ResNet50_model, topparma, ImageNetcats, cats_list[0:catlimit], 224, 224, preprocess_input, decode_predictions)
# print(ResNet50_notdogs)   

#### 利用VGG19网络进行ImageNet分类

In [ ]:
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input,decode_predictions

VGG19_model = VGG19(weights='imagenet')

# # 不是狗的
# VGG19_notdogs = get_not_animalarray(VGG19_model, topparma, ImageNetdogs, dogs_list[0:doglimit], 224, 224, preprocess_input, decode_predictions)
# print(VGG19_notdogs)

# # 不是猫的
# VGG19_notcats = get_not_animalarray(VGG19_model, topparma, ImageNetcats, cats_list[0:catlimit], 224, 224, preprocess_input, decode_predictions)
# print(VGG19_notcats)   

#### 利用Xception网络进行ImageNet分类

In [ ]:
from keras.applications.xception import Xception
from keras.applications.xception import preprocess_input,decode_predictions

Xception_model = Xception(weights='imagenet')

# # 不是狗的
# Xception_notdogs = get_not_animalarray(Xception_model, topparma, ImageNetdogs, dogs_list[0:doglimit], 299, 299, preprocess_input, decode_predictions)
# print(Xception_notdogs)

# # 不是猫的
# Xception_notcats = get_not_animalarray(Xception_model, topparma, ImageNetcats, cats_list[0:catlimit], 299, 299, preprocess_input, decode_predictions)
# print(Xception_notcats) 

#### 利用InceptionV3网络进行ImageNet分类

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input,decode_predictions

InceptionV3_model = InceptionV3(weights='imagenet')

# # 不是狗的
# InceptionV3_notdogs = get_not_animalarray(InceptionV3_model, topparma, ImageNetdogs, dogs_list[0:doglimit], 299, 299, preprocess_input, decode_predictions)
# print(InceptionV3_notdogs)

# # 不是猫的
# InceptionV3_notcats = get_not_animalarray(InceptionV3_model, topparma, ImageNetcats, cats_list[0:catlimit], 299, 299, preprocess_input, decode_predictions)
# print(InceptionV3_notcats) 

#### 合并结果集

In [ ]:
# # 合并所有的结果，取并集
# notdogs = ResNet50_notdogs + VGG19_notdogs + Xception_notdogs + InceptionV3_notdogs
# notdogs = list(set(notdogs))
# print(notdogs)

# notcats = ResNet50_notcats + VGG19_notcats + Xception_notcats + InceptionV3_notdogs
# notcats = list(set(notcats))
# print(notcats)


# notmatchs = notdogs + notcats
# print(len(notmatchs))
# # notmatchs = notdogs

#### 保存异常数据列表

In [ ]:
#保存异常数据列表
# f = open('notmatchs.txt','w')
# f.write(','.join(notmatchs))
# f.close()


#### 剔除异常数据

In [ ]:
import cv2
import  matplotlib.pyplot as plt

def show_multiple_image(images):
    '''
    展示多组图片
    '''
    count = 1
    row = len(images) / 4
    col = 4
    if (row < 1):
        row = 1
    for image in images:
        plt.figure(num='astronaut', figsize =(12,15))  #创建一个名为astronaut的窗口,并设置大小 
        img = plt.imread(image)  
        if (count == 1):
            plt.subplot(row,col,count)     #将窗口分为两行两列四个子图，则可显示四幅图片
            plt.title(image.split(splitflag)[-1])   #第一幅图片标题l
            plt.imshow(img)      #绘制第一幅图片
        else:
            plt.subplot(row,col,count)     #第三个子图
            plt.title(image.split(splitflag)[-1])   #第一幅图片标题l
            plt.imshow(img)      #绘制第一幅图片

        if (count >= 8):
            count += 1
        else:
            count += 1

    plt.show()   #显示窗口


# test1=['images/all/testnew/test\\1.jpg', 'images/all/testnew/test\\10.jpg', 'images/all/testnew/test\\100.jpg', 'images/all/testnew/test\\1000.jpg']    
# test2=['images/all/testnew/test\\1.jpg', 'images/all/testnew/test\\10.jpg', 'images/all/testnew/test\\100.jpg', 'images/all/testnew/test\\1000.jpg']    
# test3=['images/all/testnew/test\\1.jpg', 'images/all/testnew/test\\10.jpg', 'images/all/testnew/test\\100.jpg', 'images/all/testnew/test\\1000.jpg']    
# test4=['images/all/testnew/test\\101.jpg', 'images/all/testnew/test\\102.jpg', 'images/all/testnew/test\\100.jpg', 'images/all/testnew/test\\1000.jpg']    
# test5=[]
# test = test1+ test2 + test3 + test4
# print(len(notmatchs))
# show_multiple_image(notmatchs)    

In [ ]:
def read_file_as_str(file_path):
    # 判断路径文件存在
    if not os.path.isfile(file_path):
        raise TypeError(file_path + " does not exist")

    all_the_text = open(file_path).read()
    # print type(all_the_text)
    return all_the_text


def file_move(srcpath, despath):
    filenames = [item.split(splitflag)[-1] for item in sorted(srcpath)]
    count = len(filenames)
        
    for src,name in zip(srcpath,filenames):
        dst = despath + "/" + name
        if (os.path.exists(src) == True):
            shutil.move(src, dst)
            
    return count
        
notmatch_str = read_file_as_str(notmatch_txt)
notmatchs = []
notmatchs = notmatch_str.strip(',').split(',')
# print(notmatch_txt)
# print(notmatchs[0])
# print(notmatch_path)  
    

# 剔除异常数据
file_move(notmatchs, notmatch_path)

### 1.4.3 分割数据集

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.applications.inception_v3 import InceptionV3,preprocess_input
from keras.layers import GlobalAveragePooling2D,Dense
from keras.models import Model
from keras.utils.vis_utils import plot_model
from keras.optimizers import Adagrad
from sklearn.datasets import load_files
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

In [ ]:
# 加载 train 测试数据集
train_data = load_files(train_path)

# 加载 test 测试数据集
test_data = load_files(testnew_path)    

trainall_file = np.array(train_data['filenames'])
trainall_targets = np_utils.to_categorical(np.array(train_data['target']), 2)
trainall_targets = np.array(trainall_targets)[:,1]
# print(type(train_data))
print(trainall_file)
print(trainall_targets)

# 展示训练数据集
print('训练集数据总数：%d' % len(trainall_file))


# 将原来的训练数据分割成训练数据集和验证数据集，比例为20%
train_files, valid_files, mytrain_targets, myvalid_targets = train_test_split(trainall_file, trainall_targets, test_size=0.2)

print(len(train_files))
print(len(valid_files))
print(mytrain_targets)
print(myvalid_targets)


test_files = np.array(test_data['filenames'])
test_targets = np_utils.to_categorical(np.array(test_data['target']), 1)

# 展示测试数据集
print('测试集数据总数：%d' % len(test_files))

In [ ]:
from keras.preprocessing import image as image_utils
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tqdm import tqdm
from PIL import ImageFile  

# sys.modules['Image'] = Image 
# from PIL import Image
# print(Image.__file__)
# import Image
# print(Image.__file__)

# ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# def path_to_tensor(img_path):
# #     print(img_path)
#     # 用PIL加载RGB图像为PIL.Image.Image类型
#     img = image_utils.load_img(img_path,target_size=(299,299))
    
#     #将PIL.Image.Image类型转化为格式为(299,299,3)的3维张量
#     x = image_utils.img_to_array(img)
    
#     #将3维张量转化为格式为(1, 299, 299, 3) 的4维张量并返回
#     return np.expand_dims(x, axis=0)

# def paths_to_tensor(img_paths):
#     list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
#     return np.vstack(list_of_tensors)

def paths_to_tensor(img_paths):
    gen = ImageDataGenerator()
    generator = gen.flow_from_directory(img_paths, (299,299), shuffle=False, 
                                              batch_size=16)
    return generator

# img_path='images/all/train/trainnew/cats/cat.4424.jpg'
# paths_to_tensor(img_path)

In [ ]:
# from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

# img_path='images/all/trainnew/cats/cat.1584.jpg'
# img = load_img(img_path,target_size=(299,299))
# # img = Image.open(img_path)
# #将PIL.Image.Image类型转化为格式为(299,299,3)的3维张量
# x = img_to_array(img)
# print(x)

#### 归一化处理

In [ ]:
starttime = time.strftime('start time:%Y-%m-%d %H:%M:%S', time.localtime()) 
print (starttime)

# 训练集预处理
# mytrain_tensors = paths_to_tensor(train_files).astype('float32')/127.5 - 1
# print(mytrain_tensors.shape)

mytrain_tensors = paths_to_tensor(train_path)

endtime = time.strftime('end time:%Y-%m-%d %H:%M:%S', time.localtime()) 
print (endtime)

In [ ]:
starttime = time.strftime('start time:%Y-%m-%d %H:%M:%S', time.localtime()) 
print (starttime)

# 验证集预处理
# myvalid_tensors = paths_to_tensor(valid_files).astype('float32')/127.5 - 1


endtime = time.strftime('end time:%Y-%m-%d %H:%M:%S', time.localtime()) 
print (endtime)

In [ ]:
starttime = time.strftime('start time:%Y-%m-%d %H:%M:%S', time.localtime()) 
print (starttime)

# 测试集预处理
# mytest_tensors = paths_to_tensor(test_files).astype('float32')/127.5 - 1
mytest_tensors = paths_to_tensor(testnew_path)


endtime = time.strftime('end time:%Y-%m-%d %H:%M:%S', time.localtime()) 
print (endtime)

### 1.4.3 导出特征向量

In [ ]:
# 导出特征向量
def write_gap(MODEL, image_size, lambda_func=None):
    width = image_size[0]
    height = image_size[1]
    input_tensor = Input((height, width, 3))
    x = input_tensor
    if lambda_func:
        x = Lambda(lambda_func)(x)
    
    base_model = MODEL(input_tensor=x, weights='imagenet', include_top=False)
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))

    gen = ImageDataGenerator()
    train_generator = gen.flow_from_directory(train_path, image_size, shuffle=False, 
                                              batch_size=16)
    test_generator = gen.flow_from_directory(testnew_path, image_size, shuffle=False, 
                                             batch_size=16, class_mode=None)

    nb_sample = len(train_generator.filenames)
    print('nb_sample:',nb_sample)
    train = model.predict_generator(train_generator, len(train_generator.filenames), verbose=1)
    nb_sample = len(test_generator.filenames)
    test = model.predict_generator(test_generator, len(test_generator.filenames), verbose=1)
    with h5py.File("gap_%s.h5"%MODEL.func_name) as h:
        h.create_dataset("train", data=train)
        h.create_dataset("test", data=test)
        h.create_dataset("label", data=train_generator.classes)

write_gap(ResNet50, (224, 224))
write_gap(InceptionV3, (299, 299), inception_v3.preprocess_input)
write_gap(Xception, (299, 299), xception.preprocess_input)




# 2 构建模型



## 2.1 训练模型

训练 InceptionV3 模型

#### 载入特征向量

In [ ]:
#载入特征向量
train_tensors = []
test_tensors = []

for filename in ["saved_models/gap_ResNet50.h5", "saved_models/gap_Xception.h5", "saved_models/gap_InceptionV3.h5"]:
    with h5py.File(filename, 'r') as h:
        train_tensors.append(np.array(h['train']))
        test_tensors.append(np.array(h['test']))
        train_targets = np.array(h['label'])

train_tensors = np.concatenate(train_tensors, axis=1)
test_tensors = np.concatenate(test_tensors, axis=1)
train_tensors, train_targets = shuffle(train_tensors, train_targets)

# print(train_tensors)
print(train_targets)

In [ ]:
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TensorBoard
from keras.layers import Dense, Dropout, Flatten, Input


epochs = 300
batch_size = 128
patience=10

# logloss 趋势图
def show_logloss(history_callback):
    plt.plot(history_callback.history['loss'])
    plt.plot(history_callback.history['val_loss'])
    plt.title("model loss")
    plt.ylabel("loss")
    plt.xlabel("epoch")
    plt.legend(["train","test"],loc="upper left")

    plt.show()

# 准确率趋势图
def show_acc(history_callback):
    plt.plot(history_callback.history['acc'])
    plt.plot(history_callback.history['val_acc'])
    plt.title("model acc")
    plt.ylabel("acc")
    plt.xlabel("epoch")
    plt.legend(["train","test"],loc="upper left")

    plt.show()

In [ ]:
starttime = time.strftime('start time:%Y-%m-%d %H:%M:%S', time.localtime()) 
print (starttime)

# 1.构建不带分类器的预训练模型
base_model = InceptionV3(weights='imagenet',include_top=False) 

# 2.添加全局平均池化层
x = base_model.output
x = GlobalAveragePooling2D()(x)

# 3.全连接层，可选，如果精度够用则可以不加
# x = Dropout(0.5)(x)
# x = Dense(1024, activation='relu')(x)

# 4. 添加一个分类器，使用 1 个神经元，sigmoid激活函数
predictions = Dense(1, activation='sigmoid')(x)
# x = Dropout(0.5)(x)

# 5. 构建我们需要训练的完整模型
model = Model(inputs=base_model.input, outputs=predictions)


# 6.首先只训练顶部的几层（随机初始化的层），锁住所有 InceptionV3d 卷积层
for layer in base_model.layers:
    layer.trainable = False

# 7.编译模型（一定要在锁层以后操作）
# model.compile(optimizer='adadelta', loss='categorical_crossentropy',metrics=['accuracy'])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print('compile ok')


# 8. 在新的数据集上训练几代
# early stoppping 
from keras.callbacks import EarlyStopping

# monitor: 需要监视的变量
# patience:在发现变量没有变化后的多少个epoch停止
# verbose:信息展示模式
early_stopping = EarlyStopping(monitor='val_loss', patience=patience, verbose=1)

tensorboard = TensorBoard(log_dir='log')
callback_lists = [tensorboard, early_stopping]

history_callback_1 = model.fit(mytrain_tensors, mytrain_targets, batch_size=batch_size, 
                               epochs=epochs, validation_split=0.2, verbose=1, callbacks=callback_lists)


endtime = time.strftime('end time:%Y-%m-%d %H:%M:%S', time.localtime()) 
print (endtime)

In [ ]:
# base_model.summary()


In [ ]:
# model.summary()

In [ ]:
input_tensor = Input(train_tensors.shape[1:])
x = Dropout(0.5)(input_tensor)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=input_tensor, outputs=predictions)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print('compile ok')

starttime = time.strftime('start time:%Y-%m-%d %H:%M:%S', time.localtime()) 
print (starttime)

# 8. 在新的数据集上训练几代
# early stoppping 
from keras.callbacks import EarlyStopping

# monitor: 需要监视的变量
# patience:在发现变量没有变化后的多少个epoch停止
# verbose:信息展示模式
early_stopping = EarlyStopping(monitor='val_loss', patience=patience, verbose=1)

tensorboard = TensorBoard(log_dir='log')
callback_lists = [tensorboard, early_stopping]

history_callback_1 = model.fit(train_tensors, train_targets, batch_size=batch_size, 
                               epochs=epochs, validation_split=0.2, verbose=1, callbacks=callback_lists)

endtime = time.strftime('end time:%Y-%m-%d %H:%M:%S', time.localtime()) 
print (endtime)

In [ ]:
print(input_tensor)
model.summary()

In [ ]:
# 展示logloss趋势
show_logloss(history_callback_1)

In [ ]:
# 9.现在顶层应该训练好了，开始微调 InceptionV3的卷积层。
#锁住底下的几层，然后训练其余的顶层。查看每一层的名字和层号，看看应该锁多少层
for i, layer in enumerate(model.layers):
    print(i, layer.name,layer.trainable)
    
# conv2d_841 False

In [ ]:
# 10.我们选择训练最上面的两个 Inception block, 锁住前面249层，然后放开之后的层
# for layer in model.layers[:249]:
#     print('before:',layer.name,layer.trainable)
#     layer.trainable = False
# for layer in model.layers[249:]:
#     print('after:',layer.name,layer.trainable)
#     layer.trainable = True    
for layer in model.layers[:249]:
    layer.trainable = True
    print('before:',layer.name,layer.trainable)



In [ ]:
from keras.optimizers import SGD

# 11.重新编译模型，使上面的修改生效，设置一个很低的学习率:lr=0.001，使用SGD来微调
model.compile(optimizer=SGD(lr=0.0001, momentum=0., decay=0., nesterov=False), loss='binary_crossentropy', metrics=['accuracy'])

print('compile ok')

In [ ]:
starttime = time.strftime('start time:%Y-%m-%d %H:%M:%S', time.localtime()) 
print (starttime)

# 12.继续训练模型，训练最后两个 Inception block 和两个全连接层
Inceptionfile_hdf5 ='saved_models/weights.best.Inception.hdf5'

# 模型保存
checkpointer = ModelCheckpoint(filepath=Inceptionfile_hdf5, verbose=1, save_best_only=True)

# 可视化
tensorboard = TensorBoard(log_dir='log')

# 自动停止训练
# monitor: 需要监视的变量
# patience:在发现变量没有变化后的多少个epoch停止
# verbose:信息展示模式
early_stopping = EarlyStopping(monitor='val_loss', patience=patience, verbose=1)


callback_lists = [tensorboard, checkpointer, early_stopping]

# # 训练模型
# history_callback_2 = model.fit(train_tensors, train_targets, validation_split=0.2,
#          epochs=epochs, batch_size=batch_size,callbacks=callback_lists,shuffle='True',verbose=1)

history_callback_2 = model.fit(train_tensors, train_targets, batch_size=batch_size, 
                               epochs=epochs, validation_split=0.2, shuffle='True', verbose=1, callbacks=callback_lists)


endtime = time.strftime('end time:%Y-%m-%d %H:%M:%S', time.localtime()) 
print (endtime)

In [ ]:
# 展示logloss趋势
show_logloss(history_callback_2)

## 2.2 测试模型

在测试数据集上试用训练后的模型

In [ ]:
import pandas as pd
from keras.preprocessing.image import *

 

## 加载具有最好验证loss的模型
# model.load_weights(Inceptionfile_hdf5)


y_pred = model.predict(test_tensors, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)

df = pd.read_csv("sample_submission.csv")
# 
gen = ImageDataGenerator()
test_generator = gen.flow_from_directory(testnew_path, (224, 224), shuffle=False, 
                                         batch_size=16, class_mode=None)

for i, fname in enumerate(test_generator.filenames):
    index = int(fname[fname.rfind('/')+1:fname.rfind('.')])
    df.set_value(index-1, 'label', y_pred[i])

df.to_csv('pred.csv', index=None)
df.head(10)

 


In [ ]:
# from tqdm import *
# import numpy as np
# import random
# import os
# import cv2

# def get_image(index):
#     img = cv2.imread(testnew_path+'/test/%d.jpg' % index)
#     img = cv2.resize(img,(299,299))
#     img.astype(np.float32)
#     img = img / 255.0
#     return img

# test_num = 12500

# plt.figure(figsize=(12, 10))
# for i in range(20):
#     x = get_image(random.randint(1, test_num))
#     prediction = model.predict(np.expand_dims(x[0][0], axis=1))[0]
    
#     plt.subplot(4, 5, i+1)
#     if prediction < 0.5:
#         plt.title('cat %.2f%%' % (100 - prediction*100))
#     else:
#         plt.title('dog %.2f%%' % (prediction*100))
    
#     plt.axis('off')
#     plt.imshow(x[:,:,::-1]) # convert BGR to RGB

## 2.3 提交 kaggle测试得分

In [ ]:
# kaggle competitions submit -c dogs-vs-cats-redux-kernels-edition -f submission.csv -m "Message"